In [59]:
import pandas as pd
import json
import datetime
import iso3166
import ast
import numpy as np

In [2]:
e27_df_original = pd.read_csv("[e27] DataFrameB 2020-03-19.csv", index_col=0)
e27_df_original.head()

C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (16,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id_x,name_x,location_id,location_text,slug,market_x,raised_amount_x,has_been_funded,short_description,logo_thumb,...,role,headline_team_page,slug_team,logo_team,logo_thumb_team,inv_id,investor_y,Round,inv_date,Amount
0,27547.0,Alchemy Foodtech,c400,Singapore,alchemy-foodtech,"[[""Food & Beverage""]]",0.0,4.0,Diabetes has become a national focus in Singap...,https://e27.co/img/startups/27547/logo-1505806...,...,NaN,NaN,NaN,NaN,NaN,17533.0,"[{'name': 'NUS Enterprise.', 'slug': 'nus-ente...",Series A,2018-09-01,0.0
1,55341.0,Web Design World,c419,Thailand,web-design-world,"[[""Advertising"",""Information & Communications ...",0.0,0.0,"Website Design and Development, SEO and Digita...",https://e27.co/img/startups/55341/logo_thumb.jpeg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,646.0,The Wadhwa Group,NaN,NaN,NaN,NaN,0.0,0.0,NaN,https://e27.co/img/no_image.png?path=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,107.0,Indianartideas,NaN,NaN,NaN,NaN,0.0,0.0,NaN,https://e27.co/img/no_image.png?path=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,328.0,LogisticsIndonesia,NaN,NaN,NaN,NaN,0.0,0.0,NaN,https://e27.co/img/no_image.png?path=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
e27_df_original.shape

(37106, 42)

In [4]:
e27_df = e27_df_original.drop_duplicates(subset=['name_x'])

e27_df.shape

(31731, 42)

In [5]:
e27_df = e27_df.dropna(subset=['name_x'])

In [6]:
e27_df.shape

(31730, 42)

In [7]:
e27_df.isnull().sum()

id_x                        0
name_x                      0
location_id               188
location_text             190
slug                        6
market_x                 8441
raised_amount_x             0
has_been_funded             0
short_description        1063
logo_thumb                  0
website                   476
startup_id                  7
site_user_id_x              7
raising_amount           2952
fundraising              3779
monthly_traffic         30202
monthly_revenue         29670
profile_completeness       87
profile_level               7
dateadded                   7
lastupdate                  7
metas_found_month       18747
metas_found_year        18053
metas_website             477
metas_email              1812
metas_linkedin          15625
metas_twitter           16133
metas_facebook          11612
metas_investors         31715
metas_raising_round     26976
from_date_team          15258
to_date_team            15258
role                    28290
headline_t

In [8]:
def getCountryCode(country):
    try:
        code = iso3166.countries.get(country).alpha2
    except:
        code = None
    return code

sg = getCountryCode('Singapore')
sg

'SG'

In [9]:
def getLogoURL(logo_url):
    if logo_url[-4:] == '.png':
        return logo_url
    else:
        return None
    
for i in e27_df[:5]['logo_thumb']:
    print(getLogoURL(i))

https://e27.co/img/startups/27547/logo-1505806079_thumb.png
None
None
None
None


In [10]:
def getDateAdded(date):
    try:
        # example: '2017-09-19 15:26:42'
        date_stripped = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
        date_formatted = date_stripped.strftime("%Y-%m-%d")
    except:
        date_formatted = None
    
    return date_formatted

for i in e27_df[:10]['dateadded']:
    print(getDateAdded(i))

2017-09-19
None
None
None
None
None
None
None
2017-03-13
2020-03-18


In [11]:
def getFoundedDate(month, year):
    # zfill adds zero pads to single digit numbers
    try:
        found_date = str(int(year)) + "-" + str(int(month)).zfill(2) + "-01"
    except:
        found_date = None
    return found_date

for index, row in e27_df[:10].iterrows():
    print(getFoundedDate(row['metas_found_month'], row['metas_found_year']))

None
None
None
None
None
None
None
None
2018-02-01
2006-01-01


In [12]:
e27_df_original['Round'].unique()

array(['Series A', nan, 'Seed', 'Venture Round', 'Grant', 'Series C',
       'Series B', 'Pre-Seed', 'Acquisition', 'Angel', 'Series H',
       'Series F', 'Debt', 'Equity Crowdfunding', 'Series D', 'Series E',
       'Post IPO Equity', 'Series G', 'Undisclosed', 'Post IPO Debt',
       'การเข้าซื้อกิจการ', '天使'], dtype=object)

In [13]:
# structure funding round such that it fits Ninja
def printFundingRound(funding_round):
    fund = ""
    if funding_round == 'Venture Round':
        fund = 'VENTURE'
    elif funding_round == 'การเข้าซื้อกิจการ':
        fund = 'ACQUISITION'
    elif funding_round == '天使':
        fund = 'ANGEL'
    else:
        try:
            fund = funding_round.upper()
        except:
            fund = funding_round
    return fund

def getInvestors(investor_df):
    # turn the string representation of a dictionary into a real dictionary datatype
    investor_list = []
    investors = ast.literal_eval(investor_df)
    try:
        for names in investors:
            investor_name = names['name']
            investor_name = investor_name.split('(Investor)')[0].strip()
            investor_list.append(investor_name)
    except:
        pass    
    return list(set(investor_list))
    
    
def structureFunding(original_df, company_name):
    rounds = []
    try:
        sliced_df = original_df[original_df['name_x'] == company_name]
        sliced_dropdup = sliced_df.drop_duplicates(subset=['Round'])
        for index, row in sliced_dropdup.iterrows():
            round_ = {}
            try:
                round_['year'] = row['inv_date'].split('-')[0]
            except:
                round_['year'] = None
            try:
                round_['month'] = str(int(row['inv_date'].split('-')[1]))
            except:
                round_['round'] = None
            round_['amount'] = row['Amount']
            round_['currency'] = 'USD'
            round_['round'] = printFundingRound(row['Round'])
            round_['investors'] = getInvestors(row['investor_y'])
            rounds.append(round_)
    except:
        rounds = None
        
    return rounds

structureFunding(e27_df_original, 'Consultwho.sg')

#for i in structureFunding(e27_df_original, 'touchten')['Round']:
#    print(printFundingRound(i))


[{'year': '2018',
  'month': '2',
  'amount': 0.0,
  'currency': 'USD',
  'round': 'ACQUISITION',
  'investors': ['fundMyLife']}]

In [14]:
def getAllInvestors(list_of_funding_rounds):
    list_of_investors = []
    try:
        for round_ in list_of_funding_rounds:
            investors = round_['investors']
            list_of_investors.extend(investors)
    except:
        pass
    return list(set(list_of_investors))

In [15]:
example_json = {
  "id": "string",
  "lastUpdate": "2019-09-29T12:38:18+01:00",
  "name": "StudyTube",
  "tagline": "It offers experts and training organizations an online platform where they can easily create, deploy and distribute online courses",
  "about": "The most complete learning experience platform for organisations.\n         Studytube is the simplest way for organisations to find, book, manage\n         and deliver their corporate training.",
  "country": "NE",
  "founded": "2010-02-21",
  "profileUrl": "https://www.linkedin.com/company/studytube",
  "domain": "studytube.com",
  "linkedinUrl": "https://www.linkedin.com/company/studytube",
  "crunchbaseUrl": "http://www.crunchbase.com/company/studytube",
  "logo": "https://www.studytube.nl/hubfs/Studytube-2018/images/logo.svg",
  "headcounts": [
    {
      "when": "2019-11-21",
      "count": "integer",
      "range": "51-500"
    }
  ],
  "personsOfInterest": [
    {
      "name": "Homam Karimi",
      "title": "CEO, Founder",
      "image": "https://s3.eu-west-2.amazonaws.com/dealroom-static/null/people-100.png",
      "profiles": {
        "linkedin": "https://www.linkedin.com/in/homam-karimi-45813915",
        "crunchbase": "",
        "dealroom": ""
      },
      "bio": "Homam is an entrepreneur in the field of education. He is the founder and CEO of StudyTube, the one stop solution for organizations to mange and develop the competencies of their employees, more efficiently. "
    }
  ],
  "specialities": [
    "education"
  ],
  "rounds": [
    {
      "year": 2017,
      "month": 2,
      "amount": 2000000,
      "currency": "EUR",
      "round": "SERIES B",
      "investors": [
        "Vortex Capital Partners"
      ]
    }
  ],
  "investors": [
    "Vortex Capital Partners"
  ],
  "additionalData": "object"
}

example_json

{'id': 'string',
 'lastUpdate': '2019-09-29T12:38:18+01:00',
 'name': 'StudyTube',
 'tagline': 'It offers experts and training organizations an online platform where they can easily create, deploy and distribute online courses',
 'about': 'The most complete learning experience platform for organisations.\n         Studytube is the simplest way for organisations to find, book, manage\n         and deliver their corporate training.',
 'country': 'NE',
 'founded': '2010-02-21',
 'profileUrl': 'https://www.linkedin.com/company/studytube',
 'domain': 'studytube.com',
 'linkedinUrl': 'https://www.linkedin.com/company/studytube',
 'crunchbaseUrl': 'http://www.crunchbase.com/company/studytube',
 'logo': 'https://www.studytube.nl/hubfs/Studytube-2018/images/logo.svg',
 'headcounts': [{'when': '2019-11-21', 'count': 'integer', 'range': '51-500'}],
 'personsOfInterest': [{'name': 'Homam Karimi',
   'title': 'CEO, Founder',
   'image': 'https://s3.eu-west-2.amazonaws.com/dealroom-static/null/peopl

In [16]:
e27_data = {}

for index, row in e27_df[20:25].iterrows():
    e27_data['id'] = 'string'
    e27_data['lastUpdate'] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S+00:00")
    e27_data['name'] = row['name_x']
    e27_data['tagline'] = row['short_description']
    #e27_data['about'] = row['metas_description']
    e27_data['country'] = getCountryCode(row['location_text'])
    e27_data['founded'] = getFoundedDate(row['metas_found_month'], row['metas_found_year'])
    #e27_data['profileUrl'] = 
    e27_data['domain'] = row['website']
    e27_data['linkedinUrl'] = row['metas_linkedin']
    #e27['crunchbaseUrl'] = 
    e27_data['logo'] = getLogoURL(row['logo_thumb'])
    #e27_data['headcounts'] =
    #e27_data['specialities'] = 
    e27_data['rounds'] = structureFunding(e27_df_original, row['name_x'])
    e27_data['investors'] = getAllInvestors(e27_data['rounds'])
    print(e27_data)
    print('')

{'id': 'string', 'lastUpdate': '2020-03-25T19:58:05+00:00', 'name': 'ELLA STUDY', 'tagline': 'An online platform connecting Vietnamese students with alumni for overseas universities placement consultations.', 'country': None, 'founded': None, 'domain': 'https://ellastudy.com/', 'linkedinUrl': nan, 'logo': 'https://e27.co/img/startups/26597/logo-1502421138_thumb.png', 'rounds': [{'year': '2017', 'month': '10', 'amount': 30000.0, 'currency': 'USD', 'round': 'SEED', 'investors': ['G&H Ventures', 'VIISA']}], 'investors': ['VIISA', 'G&H Ventures']}

{'id': 'string', 'lastUpdate': '2020-03-25T19:58:05+00:00', 'name': 'MOJITOK', 'tagline': 'The startup offers third-party smart AI that helps read emotions and converts text to stickers.', 'country': None, 'founded': None, 'domain': 'http://www.mojitok.com/', 'linkedinUrl': nan, 'logo': 'https://e27.co/img/startups/26596/logo-1502420176_thumb.png', 'rounds': [{'year': '2017', 'month': '10', 'amount': 15000.0, 'currency': 'USD', 'round': 'SEED', 

In [17]:
e27_data

{'id': 'string',
 'lastUpdate': '2020-03-25T19:58:05+00:00',
 'name': 'UseData  ',
 'tagline': 'UseData offers a big data analytics platform for E-commerce businesses with ability to customize their processing needs.',
 'country': None,
 'founded': None,
 'domain': 'http://usedata.vn/',
 'linkedinUrl': 'https://www.linkedin.com/company/usedata',
 'logo': 'https://e27.co/img/startups/7475/logo-1450335213_thumb.png',
 'rounds': [{'year': '2017',
   'month': '4',
   'amount': 30000.0,
   'currency': 'USD',
   'round': 'SEED',
   'investors': ['VIISA']}],
 'investors': ['VIISA']}

In [24]:
import requests

in_ninja = 0
out_ninja = 0 

#for name_source in data_source['Company Name']:
#for name_source in e27_df['name_x']:    
    #print(name_source)
    url = 'https://i8i7s543g6.execute-api.us-west-2.amazonaws.com/production/connector/reconcile'
    data = {"name": name_source, "type":"COMPANY"}
    api_token = ("x-api-key" , "v6ZBsS9BK59GlP7O7Eb3x1WHQ83spOlj4mDIk3c9")
    # payload = open("request.json")
    headers = {"Content-Type" : "application/json", "Content-Type" : "application/x-www-form-urlencoded", 
               "x-api-key" : "v6ZBsS9BK59GlP7O7Eb3x1WHQ83spOlj4mDIk3c9"}
    r = requests.post(url, json = data, headers=headers)
    #print(r.text)
    if len(r.text)==2:
        out_ninja += 1
        #print("out_ninja" + str(out_ninja))
    elif len(r.text) > 2:
        in_ninja += 1
        #print("in_ninja" + str(in_ninja))
    else:
        print(r.text)
        
print(out_ninja)
print(in_ninja)

21609
10121


In [2]:
import pandas as pd
main_data = pd.read_csv(r'C:\Users\User\Desktop\Fresh\Vertex\GlassDeed_Companies_Full_List.csv')

In [3]:
import requests

url = 'https://i8i7s543g6.execute-api.us-west-2.amazonaws.com/production/connector/reconcile'
api_token = ("x-api-key" , "v6ZBsS9BK59GlP7O7Eb3x1WHQ83spOlj4mDIk3c9")
headers = {"Content-Type" : "application/json", "Content-Type" : "application/x-www-form-urlencoded", 
               "x-api-key" : "v6ZBsS9BK59GlP7O7Eb3x1WHQ83spOlj4mDIk3c9"}

def confirmNinjaCompanies(company_name):
    isPresent = []
    data = {"name": company_name, "type":"COMPANY"}
    r = requests.post(url, json = data, headers=headers)
    if len(r.text)==2:
        isPresent.append(0)
    elif len(r.text) > 2:
        isPresent.append(1)
    else:
        isPresent.append(None)
    temp_df = pd.DataFrame({"name_x": [company_name], "isPresent": isPresent})
    return temp_df

In [4]:
main_data['Company Name']

0                                           KPMG
1                               Telekom Malaysia
2                                        Foxconn
3                                      foodpanda
4                                       Flipkart
                          ...                   
67320    compagnie des transports strasbourgeois
67321                    valley hope association
67322                              auchan retail
67323                                   prodware
67324                             lastminute.com
Name: Company Name, Length: 67325, dtype: object

In [33]:
confirmNinjaCompanies('Crowde')

,name_x,isPresent
0,Crowde,1


In [5]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

with ThreadPoolExecutor(max_workers=10) as executor:
    start = time.time()
    ninja_results = []
    futures = [executor.submit(confirmNinjaCompanies, company_name) for company_name in main_data['Company Name']]
    for result in as_completed(futures):
        ninja_results.append(result)
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))

Time Taken: 2467.754298s


In [6]:
list_of_ninja_results = []

for i in ninja_results:
    list_of_ninja_results.append(i.result())
    
combined_ninja_status = pd.concat(list_of_ninja_results)
combined_ninja_status

,name_x,isPresent
0,Fire Moon Studios,1
0,Yondu,1
0,Testbook.com,1
0,Telekom Malaysia,1
0,MediaTek,0
...,...,...
0,omega pharma,1
0,valley hope association,0
0,auchan retail,1
0,lastminute.com,1


In [7]:
combined_ninja_status = combined_ninja_status.reset_index(drop=True)

In [8]:
combined_ninja_status['isPresent'].value_counts()

0    36148
1    31177
Name: isPresent, dtype: int64

In [10]:
main_data.columns

Index(['Unnamed: 0', 'Company Name', 'Headquarters', 'Size', 'Industry',
       'Revenue', 'Description', 'CEO Approval', 'Company Benefits',
       'Culture Values', 'Career Opportunities', 'Work Life Balance',
       'Senior Management Approval', 'Rating', 'URL'],
      dtype='object')

In [30]:
combined_ninja_status = combined_ninja_status.rename(columns={'name_x':'Company Name', 'isPresent':'In_Ninja'})

In [31]:
df_v2 = pd.merge(main_data, combined_ninja_status, on='Company Name')
df_v2

,Unnamed: 0,Company Name,Headquarters,Size,Industry,Revenue,Description,CEO Approval,Company Benefits,Culture Values,Career Opportunities,Work Life Balance,Senior Management Approval,Rating,URL,In_Ninja
0,0,KPMG,"KPMG France, Tour Eqho, 2 avenue Gambetta 9240...","10,000+",Consulting and Business Services,$1B to $5B (USD),KPMG is among the world’s leading professional...,88%,3.6,3.8,3.8,3.4,3.6,4.0,NaN,1
1,9,Telekom Malaysia,"Telekom Tower, Kuala Lumpur, Malaysia",NaN,NaN,NaN,"Telekom Malaysia Berhad (MYX: 4863), DBA TM, i...",86%,3.7,4.0,3.8,4.0,3.8,4.1,NaN,1
2,37,Foxconn,"Taipei, TW",201 to 500,Industrial Manufacturing,$1B to $5B (USD),"Hon Hai Precision Industry Co., Ltd., trading ...",84%,3.9,3.7,3.4,3.9,3.6,4.0,NaN,0
3,40,foodpanda,Berlin,"5,001 to 10,000",Internet and Software,less than $1M (USD),The foodpanda group is one of the world’s larg...,79%,3.5,3.6,3.2,3.7,3.3,3.8,NaN,0
4,44,Flipkart,Bengaluru,"10,000+",Internet and Software,$5B to $10B (USD),Flipkart is an e-commerce company founded in 2...,78%,3.7,4.0,3.6,3.9,3.7,4.1,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67320,343101,compagnie des transports strasbourgeois,"Strasbourg, FR","1,001 to 5,000",NaN,$25M to $100M (USD),The Compagnie des Transports Strasbourgeois (C...,NaN,3.6,3.2,3.6,3.2,3.2,3.6,https://www.cts-strasbourg.eu/,0
67321,345608,valley hope association,"P.O. Box 510 Norton, Kansas 67654","501 to 1,000",Health Care,NaN,Valley Hope opened its doors in 1967 in the sm...,67%,3.1,3.2,2.8,3.2,2.9,3.4,"https://twitter.com/valleyhopeassoc, https://w...",0
67322,347630,auchan retail,"200, rue de la Recherche 59491 VILLENEUVE D'ASCQ","10,000+",Retail,NaN,Auchan Retail recrute en ce moment.,50%,3.4,3.6,3.2,3.5,3.3,3.7,"https://twitter.com/AuchanRecrute, https://www...",1
67323,353921,prodware,"Paris, FR","501 to 1,000",NaN,$100M to $500M (USD),NaN,NaN,3.3,3.5,3.2,3.4,3.3,3.4,http://www.prodware.fr/,1


In [32]:
def check_LinkedIn(string):
    if 'linkedin' in string:
        return 1
    else:
        return 0

In [33]:
df_v2['LinkedIn present'] = df_v2['URL'].astype(str).str.lower().apply(check_LinkedIn)

In [35]:
df_v2.to_csv(r'C:\Users\User\Desktop\Fresh\Vertex\Ninja_check.csv')

In [60]:
ninjaYesLinkedInYes = []
ninjaYesLinkedInNo = []
ninjaNoLinkedInYes = []
ninjaNoLinkedInNo = []

for index, row in e27_df_v2[:10].iterrows():
    if row['metas_linkedin'] is not np.nan:
        if row['isPresent'] == 1:
            ninjaYesLinkedInYes.append(row['name_x'])
        else:
            ninjaNoLinkedInYes.append(row['name_x'])
    else:
        if row['isPresent'] == 1:
            ninjaYesLinkedInNo.append(row['name_x'])
        else:
            ninjaNoLinkedInNo.append(row['name_x'])


In [61]:
ninjaYesLinkedInYes

['ewazte', 'ByteAnt']

In [62]:
ninjaYesLinkedInNo

['The Wadhwa Group', 'Indianartideas', 'EZY-Booking.com']

In [63]:
ninjaNoLinkedInYes

[]

In [64]:
ninjaNoLinkedInNo

['Alchemy Foodtech',
 'Web Design World',
 'LogisticsIndonesia',
 'Ads2Publish.com',
 'Mondano']

In [65]:
e27_df_v2[:10]['metas_linkedin']

0                                         NaN
1                                         NaN
2                                         NaN
3                                         NaN
4                                         NaN
5                                         NaN
6                                         NaN
7                                         NaN
8    https://www.linkedin.com/company/ewazte/
9    https://www.linkedin.com/company/byteant
Name: metas_linkedin, dtype: object

In [66]:
e27_df_v2[:10]['isPresent']

0    0
1    0
2    1
3    1
4    0
5    0
6    0
7    1
8    1
9    1
Name: isPresent, dtype: int64

In [67]:
ninjaYesLinkedInYes = []
ninjaYesLinkedInNo = []
ninjaNoLinkedInYes = []
ninjaNoLinkedInNo = []

for index, row in e27_df_v2.iterrows():
    if row['metas_linkedin'] is not np.nan:
        if row['isPresent'] == 1:
            ninjaYesLinkedInYes.append(row['name_x'])
        else:
            ninjaNoLinkedInYes.append(row['name_x'])
    else:
        if row['isPresent'] == 1:
            ninjaYesLinkedInNo.append(row['name_x'])
        else:
            ninjaNoLinkedInNo.append(row['name_x'])

print(len(ninjaYesLinkedInYes))
print(len(ninjaYesLinkedInNo))
print(len(ninjaNoLinkedInYes))
print(len(ninjaNoLinkedInNo))

7156
2956
8949
12669
